# Video Identification of Suspicious Behavior: Preparation

This notebook will **prepare** your video data by:
* Processing the images by extracting out individual images (using `open-cv`) and saving them to DBFS / cloud storage
* With the saved images, execute Spark Deep Learning Pipelines `DeepImageFeaturizer` to extract image features and saving them to DBFS / cloud storage in Parquet format
 * Perform this task for both the training and test datasets

The source data used in this notebook can be found at [EC Funded CAVIAR project/IST 2001 37540](http://homepages.inf.ed.ac.uk/rbf/CAVIAR/)

<img src="https://databricks.com/wp-content/uploads/2018/09/mnt_raela_video_splash.png" width=900/>


### Cluster Configuration
* Suggested cluster configuration:
 * Databricks Runtime Version: `Databricks Runtime for ML` (e.g. 4.1 ML, 4.2 ML, etc.)
 * Driver: 64GB RAM Instance (e.g. `Azure: Standard_D16s_v3`)
 * Workers: 2x 64GB RAM Instance (e.g. `Azure: Standard_D16s_v3`)
 * Python: `Python 3`
 
### Need to install manually
To install, refer to **Upload a Python PyPI package or Python Egg** [Databricks](https://docs.databricks.com/user-guide/libraries.html#upload-a-python-pypi-package-or-python-egg) | [Azure Databricks](https://docs.azuredatabricks.net/user-guide/libraries.html#upload-a-python-pypi-package-or-python-egg)

* Python Libraries:
 * `opencv-python`: 3.4.2 
 
### Libraries Already Included in Databricks Runtime for ML
Because we're using *Databricks Runtime for ML*, you do **not** need to install the following libraires
* Python Libraries:
 * `h5py`: 2.7.1
 * `tensorflow`: 1.7.1
 * `keras`: 2.1.5 (Using TensorFlow backend)
 * *You can check by `import tensorflow as tf; print(tf.__version__)`*

* JARs:
 * `spark-deep-learning-1.0.0-spark2.3-s_2.11.jar`
 * `tensorframes-0.3.0-s_2.11.jar`
 * *You can check by reviewing cluster's Spark UI > Environment)*

In [2]:
  import cv2
  import uuid
  import re
  import glob
  import os
  
  MY_NAME = 'silinskiy' + "/"

In [3]:
srcVideoPath = "/databricks-datasets/cctvVideos/train/"
srcTestVideoPath =  "/databricks-datasets/cctvVideos/test/"
targetImgPath = "/mnt/tardis6/videos/cctvFrames/train-" + MY_NAME + "/"
targetImgTestPath = "/mnt/tardis6/videos/cctvFrames/test-" + MY_NAME + "/"
imgFeaturesPath = "/mnt/tardis6/videos/cctv_features/train-" + MY_NAME + "/"
imgFeaturesTestPath = "/mnt/tardis6/videos/cctv_features/test-" + MY_NAME + "/"
labeledDataPath = "/databricks-datasets/cctvVideos/labels/"
srcVideoMP4Path = "/databricks-datasets/cctvVideos/mp4/train/"
srcTestVideoMP4Path = "/databricks-datasets/cctvVideos/mp4/test/"

# displayVid(): Shows video from mounted cloud storage
def displayVid(filepath):
  return displayHTML("""
  <video width="480" height="320" controls>
  <source src="/files/%s" type="video/mp4">
  </video>
  """ % filepath)

# displayDbfsVid(): Shows video from DBFS
def displayDbfsVid(filepath):
  return displayHTML("""
  <video width="480" height="320" controls>
  <source src="/dbfs/%s" type="video/mp4">
  </video>
  """ % filepath)

# displayImg(): Shows image from dbfs/cloud storage
def displayImg(filepath):
  dbutils.fs.cp(filepath, "FileStore/%s" % filepath)
  return displayHTML("""
  <img src="/files/%s">
  """ % filepath)

In [4]:
display(dbutils.fs.ls(srcVideoPath))

path,name,size
dbfs:/databricks-datasets/cctvVideos/train/Browse2.mpg,Browse2.mpg,10005185
dbfs:/databricks-datasets/cctvVideos/train/Browse3.mpg,Browse3.mpg,10503996
dbfs:/databricks-datasets/cctvVideos/train/Browse4.mpg,Browse4.mpg,13152710
dbfs:/databricks-datasets/cctvVideos/train/Browse_WhileWaiting1.mpg,Browse_WhileWaiting1.mpg,9033442
dbfs:/databricks-datasets/cctvVideos/train/Fight_Chase.mpg,Fight_Chase.mpg,5087190
dbfs:/databricks-datasets/cctvVideos/train/Fight_RunAway1.mpg,Fight_RunAway1.mpg,6500613
dbfs:/databricks-datasets/cctvVideos/train/LeftBag.mpg,LeftBag.mpg,17201025
dbfs:/databricks-datasets/cctvVideos/train/LeftBox.mpg,LeftBox.mpg,10185955
dbfs:/databricks-datasets/cctvVideos/train/Rest_WiggleOnFloor.mpg,Rest_WiggleOnFloor.mpg,15097843
dbfs:/databricks-datasets/cctvVideos/train/Walk1.mpg,Walk1.mpg,7208095


Here is an example video that we will perform our training on to identify suspicious behavior.
* The source of this data is from the [EC Funded CAVIAR project/IST 2001 37540](http://homepages.inf.ed.ac.uk/rbf/CAVIAR/).

![](https://databricks.com/wp-content/uploads/2018/09/Browse2.gif)

Extract JPG images from MPG videos using OpenCV (`cv2`)

In [7]:
# Extract and Save Images using CV2
def extractImagesSave(src, tgt):
  
  # Ensure directory is created
  dbutils.fs.mkdirs(tgt)

  ## Extract one video frame per second and save frame as JPG
  def extractImages(pathIn):
    count = 0
    srcVideos = "/dbfs" + src + "(.*).mpg"
    p = re.compile(srcVideos)
    vidName = str(p.search(pathIn).group(1))
    vidcap = cv2.VideoCapture(pathIn)
    success,image = vidcap.read()
    success = True
    while success:
      vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*1000))
      success,image = vidcap.read()
      print ('Read a new frame: ', success)
      cv2.imwrite("/dbfs" + tgt + vidName + "frame%04d.jpg" % count, image)     # save frame as JPEG file
      count = count + 1
      print ('Wrote a new frame')
        
  ## Extract frames from all videos and save in folder
  def createFUSEpaths(dbfsFilePath):
    return "/dbfs/" + dbfsFilePath[0][6:]
  
  # Build up fileList RDD
  fileList = dbutils.fs.ls(src)
  FUSEfileList = map(createFUSEpaths, fileList)
  FUSEfileList_rdd = sc.parallelize(FUSEfileList)
  
  # Extract and save images
  FUSEfileList_rdd.map(extractImages).count()

  
# Remove Empty Files
def removeEmptyFiles(pathDir):
  import sys
  import os

  rootDir = '/dbfs' + pathDir
  for root, dirs, files in os.walk(rootDir):
    for f in files:
      fileName = os.path.join(root, f)
      if os.path.getsize(fileName) == 0:
        print ("empty fileName: %s \n" % fileName)
        os.remove(fileName)


# Training Dataset

In [9]:
# Extract Images
extractImagesSave(srcVideoPath, targetImgPath)

# Remove Empty Files
removeEmptyFiles(targetImgPath)

# View file list of images extracted from video
display(dbutils.fs.ls(targetImgPath))

path,name,size
dbfs:/mnt/tardis6/videos/cctvFrames/train-silinskiy/Browse2frame0000.jpg,Browse2frame0000.jpg,45025
dbfs:/mnt/tardis6/videos/cctvFrames/train-silinskiy/Browse2frame0001.jpg,Browse2frame0001.jpg,52545
dbfs:/mnt/tardis6/videos/cctvFrames/train-silinskiy/Browse2frame0002.jpg,Browse2frame0002.jpg,52801
dbfs:/mnt/tardis6/videos/cctvFrames/train-silinskiy/Browse2frame0003.jpg,Browse2frame0003.jpg,53165
dbfs:/mnt/tardis6/videos/cctvFrames/train-silinskiy/Browse2frame0004.jpg,Browse2frame0004.jpg,53307
dbfs:/mnt/tardis6/videos/cctvFrames/train-silinskiy/Browse2frame0005.jpg,Browse2frame0005.jpg,53787
dbfs:/mnt/tardis6/videos/cctvFrames/train-silinskiy/Browse2frame0006.jpg,Browse2frame0006.jpg,45644
dbfs:/mnt/tardis6/videos/cctvFrames/train-silinskiy/Browse2frame0007.jpg,Browse2frame0007.jpg,49379
dbfs:/mnt/tardis6/videos/cctvFrames/train-silinskiy/Browse2frame0008.jpg,Browse2frame0008.jpg,50835
dbfs:/mnt/tardis6/videos/cctvFrames/train-silinskiy/Browse2frame0009.jpg,Browse2frame0009.jpg,51129


In [10]:
trainImages = spark.read.format("image").load(targetImgPath)
display(trainImages)

image
"List(dbfs:/mnt/tardis6/videos/cctvFrames/train-silinskiy/LeftBagframe0004.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/train-silinskiy/LeftBagframe0040.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/train-silinskiy/LeftBagframe0005.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/train-silinskiy/LeftBagframe0015.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/train-silinskiy/LeftBagframe0017.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/train-silinskiy/LeftBagframe0003.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/train-silinskiy/LeftBoxframe0016.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/train-silinskiy/LeftBagframe0053.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/train-silinskiy/LeftBagframe0041.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/train-silinskiy/LeftBagframe0002.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"


Use [Spark Deep Learning Pipelines](https://github.com/databricks/spark-deep-learning) `DeepImageFeaturizer` to build image features via the InceptionV3 model

In [12]:
# Save Image Features using 
def saveImageFeatures(images, filePath):
  from sparkdl import DeepImageFeaturizer

  # Build featurizer using DeepImageFeaturizer and the InceptionV3 model
  featurizer = DeepImageFeaturizer(inputCol="image", outputCol="features", modelName="InceptionV3")

  # Transform images to pull out image (origin, height, width, nChannels, mode, data) and features (udt)
  features = featurizer.transform(images)

  # Push feature information into Parquet file format
  # This might take a few minutes
  dbutils.fs.mkdirs(filePath)

  # Extract only image file name (imgFileName) within our saved features
  features.select("image.origin", "features").coalesce(2).write.mode("overwrite").parquet(filePath)

In [13]:
saveImageFeatures(trainImages, imgFeaturesPath)

/databricks/python/lib/python3.7/site-packages/botocore/vendored/requests/packages/urllib3/_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 from collections import Mapping, MutableMapping
Using TensorFlow backend.

In [14]:
# View Parquet Features
display(dbutils.fs.ls(imgFeaturesPath))

path,name,size
dbfs:/mnt/tardis6/videos/cctv_features/train-silinskiy/_SUCCESS,_SUCCESS,0
dbfs:/mnt/tardis6/videos/cctv_features/train-silinskiy/_committed_2421411134247649386,_committed_2421411134247649386,224
dbfs:/mnt/tardis6/videos/cctv_features/train-silinskiy/_started_2421411134247649386,_started_2421411134247649386,0
dbfs:/mnt/tardis6/videos/cctv_features/train-silinskiy/part-00000-tid-2421411134247649386-2ce11d66-db26-482b-a6af-93f1c91f4f6d-486-1-c000.snappy.parquet,part-00000-tid-2421411134247649386-2ce11d66-db26-482b-a6af-93f1c91f4f6d-486-1-c000.snappy.parquet,88537741
dbfs:/mnt/tardis6/videos/cctv_features/train-silinskiy/part-00001-tid-2421411134247649386-2ce11d66-db26-482b-a6af-93f1c91f4f6d-487-1-c000.snappy.parquet,part-00001-tid-2421411134247649386-2ce11d66-db26-482b-a6af-93f1c91f4f6d-487-1-c000.snappy.parquet,87525165


# Test Dataset

In [16]:
display(dbutils.fs.ls(srcTestVideoPath))

path,name,size
dbfs:/databricks-datasets/cctvVideos/test/Browse1.mpg,Browse1.mpg,12405350
dbfs:/databricks-datasets/cctvVideos/test/Browse_WhileWaiting2.mpg,Browse_WhileWaiting2.mpg,21395638
dbfs:/databricks-datasets/cctvVideos/test/Fight_OneManDown.mpg,Fight_OneManDown.mpg,11320119
dbfs:/databricks-datasets/cctvVideos/test/Fight_RunAway2.mpg,Fight_RunAway2.mpg,6483410
dbfs:/databricks-datasets/cctvVideos/test/LeftBag_AtChair.mpg,LeftBag_AtChair.mpg,12904113
dbfs:/databricks-datasets/cctvVideos/test/LeftBag_BehindChair.mpg,LeftBag_BehindChair.mpg,12593939
dbfs:/databricks-datasets/cctvVideos/test/LeftBox.mpg,LeftBox.mpg,10185955
dbfs:/databricks-datasets/cctvVideos/test/Rest_SlumpOnFloor.mpg,Rest_SlumpOnFloor.mpg,10634883
dbfs:/databricks-datasets/cctvVideos/test/Walk3.mpg,Walk3.mpg,15889216


In [17]:
# Extract Images
extractImagesSave(srcTestVideoPath, targetImgTestPath)

# Remove Empty Files
removeEmptyFiles(targetImgTestPath)

# View file list of images extracted from video
display(dbutils.fs.ls(targetImgTestPath))

path,name,size
dbfs:/mnt/tardis6/videos/cctvFrames/test-silinskiy/Browse1frame0000.jpg,Browse1frame0000.jpg,45922
dbfs:/mnt/tardis6/videos/cctvFrames/test-silinskiy/Browse1frame0001.jpg,Browse1frame0001.jpg,54227
dbfs:/mnt/tardis6/videos/cctvFrames/test-silinskiy/Browse1frame0002.jpg,Browse1frame0002.jpg,53948
dbfs:/mnt/tardis6/videos/cctvFrames/test-silinskiy/Browse1frame0003.jpg,Browse1frame0003.jpg,53802
dbfs:/mnt/tardis6/videos/cctvFrames/test-silinskiy/Browse1frame0004.jpg,Browse1frame0004.jpg,53689
dbfs:/mnt/tardis6/videos/cctvFrames/test-silinskiy/Browse1frame0005.jpg,Browse1frame0005.jpg,54099
dbfs:/mnt/tardis6/videos/cctvFrames/test-silinskiy/Browse1frame0006.jpg,Browse1frame0006.jpg,45806
dbfs:/mnt/tardis6/videos/cctvFrames/test-silinskiy/Browse1frame0007.jpg,Browse1frame0007.jpg,49294
dbfs:/mnt/tardis6/videos/cctvFrames/test-silinskiy/Browse1frame0008.jpg,Browse1frame0008.jpg,49152
dbfs:/mnt/tardis6/videos/cctvFrames/test-silinskiy/Browse1frame0009.jpg,Browse1frame0009.jpg,51930


In [18]:
testImages = spark.read.format("image").load(targetImgTestPath)
display(testImages)

image
"List(dbfs:/mnt/tardis6/videos/cctvFrames/test-silinskiy/LeftBoxframe0016.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/test-silinskiy/LeftBag_BehindChairframe0005.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/test-silinskiy/LeftBag_BehindChairframe0040.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/test-silinskiy/LeftBag_AtChairframe0003.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/test-silinskiy/LeftBag_BehindChairframe0041.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/test-silinskiy/Rest_SlumpOnFloorframe0005.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/test-silinskiy/LeftBoxframe0005.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/test-silinskiy/LeftBoxframe0015.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/test-silinskiy/Browse1frame0001.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/test-silinskiy/LeftBoxframe0025.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"


In [19]:
saveImageFeatures(testImages, imgFeaturesTestPath)

In [20]:
# View Parquet Features
display(dbutils.fs.ls(imgFeaturesTestPath))

path,name,size
dbfs:/mnt/tardis6/videos/cctv_features/test-silinskiy/_SUCCESS,_SUCCESS,0
dbfs:/mnt/tardis6/videos/cctv_features/test-silinskiy/_committed_7052309044295673012,_committed_7052309044295673012,226
dbfs:/mnt/tardis6/videos/cctv_features/test-silinskiy/_started_7052309044295673012,_started_7052309044295673012,0
dbfs:/mnt/tardis6/videos/cctv_features/test-silinskiy/part-00000-tid-7052309044295673012-a5bc7c52-8314-4326-8b6a-b59682d33023-1588-1-c000.snappy.parquet,part-00000-tid-7052309044295673012-a5bc7c52-8314-4326-8b6a-b59682d33023-1588-1-c000.snappy.parquet,86984112
dbfs:/mnt/tardis6/videos/cctv_features/test-silinskiy/part-00001-tid-7052309044295673012-a5bc7c52-8314-4326-8b6a-b59682d33023-1589-1-c000.snappy.parquet,part-00001-tid-7052309044295673012-a5bc7c52-8314-4326-8b6a-b59682d33023-1589-1-c000.snappy.parquet,85614947


In [21]:
# Set the spark config for direct connection to Azure Blob Storage with the Account key found in Azure Portal
spark.conf.set( "fs.azure.account.key.aicampsilinskiy.blob.core.windows.net", "FRVMHxU1SOiZQ2toFkEJd054E6tisMC0p3csho9D+bkJjfEcGrDWK52FeXmMvwnBqTlyblo4q7slv5Ktk5UCBg==")

# # Create list of extracted training images on dbfs
# train_images = glob.glob("/dbfs" + targetImgPath + "/*.jpg")
# print(len(train_images))

# # Copy images to blob storage, could also zipped and copied as one file
# for img in train_images:
#   dbutils.fs.cp(img.replace("/dbfs", ""), "wasbs://cvcamp@michhargeneralstore.blob.core.windows.net/" + img.replace("/dbfs/mnt/tardis6/videos/", ""))

# # Create list of extracted training images on dbfs
# test_images = glob.glob("/dbfs" + targetImgTestPath + "/*.jpg")
# print(len(test_images))

# # Copy images to blob storage, could also zipped and copied as one file
# for img in test_images:
#   dbutils.fs.cp(img.replace("/dbfs", ""), "wasbs://cvcamp@michhargeneralstore.blob.core.windows.net/" + img.replace("/dbfs/mnt/tardis6/videos/", ""))

dbutils.fs.cp("/databricks-datasets/cctvVideos/labels/cctvFrames_train_labels.csv", "wasbs://tardis1@aicampsilinskiy.blob.core.windows.net/labels/cctvFrames_train_labels.csv")

Out[25]: True

In [22]:
import shutil

shutil.make_archive('train-for-upload', 'zip', '/dbfs/mnt/tardis6/videos/cctvFrames/', 'train-' + MY_NAME)
shutil.make_archive('test-for-upload', 'zip', '/dbfs/mnt/tardis6/videos/cctvFrames/', 'test-' + MY_NAME)

Out[26]: '/databricks/driver/test-for-upload.zip'

In [23]:
dbutils.fs.ls("file:/databricks/driver/")

Out[27]: [FileInfo(path='file:/databricks/driver/conf/', name='conf/', size=4096),
 FileInfo(path='file:/databricks/driver/logs/', name='logs/', size=4096),
 FileInfo(path='file:/databricks/driver/derby.log', name='derby.log', size=729),
 FileInfo(path='file:/databricks/driver/eventlogs/', name='eventlogs/', size=4096),
 FileInfo(path='file:/databricks/driver/train-for-upload.zip', name='train-for-upload.zip', size=20655269),
 FileInfo(path='file:/databricks/driver/ganglia/', name='ganglia/', size=4096),
 FileInfo(path='file:/databricks/driver/test-for-upload.zip', name='test-for-upload.zip', size=20165331)]

In [24]:
dbutils.fs.cp("file:/databricks/driver/train-for-upload.zip", "wasbs://tardis1@aicampsilinskiy.blob.core.windows.net/train.zip")
dbutils.fs.cp("file:/databricks/driver/test-for-upload.zip", "wasbs://tardis1@aicampsilinskiy.blob.core.windows.net/test.zip")

Out[28]: True

In [25]:
# shutil.rmtree('file:/databricks/driver/train-for-upload.zip')
# os.remove('file:/databricks/driver/train-for-upload.zip')